In [2]:
import os
import torch

print(os.listdir('.'))
print(os.getcwd())
os.chdir('/home/robsyc/Desktop/thesis/MB-VAE-DTI/')

print(torch.cuda.is_available())
torch.cuda.empty_cache()

['data', 'bmfm_sm', 'models', '.gitignore', 'jobs', 'notebooks', 'requirements.txt']
/home/robsyc/Desktop/thesis/MB-VAE-DTI
True


# Load Example Data

In [3]:
import h5torch

dataset = h5torch.Dataset(
    "./data/DAVIS.h5t",
    sampling="coo",
    in_memory=True,
)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=12, shuffle=True)
batch = next(iter(dataloader))
print(batch["0/Drug_SMILES"])
print(batch["1/Target_seq"])

['CC(C)N1NC(=C2C=c3cc(O)ccc3=N2)c2c(N)ncnc21', 'CN(C)CC1CCn2cc(c3ccccc32)C2=C(C(=O)NC2=O)c2cn(c3ccccc23)CCO1', 'CC(C)N1NC(=C2C=c3cc(O)ccc3=N2)c2c(N)ncnc21', 'CC(C)(C)c1cnc(CSc2cnc(NC(=O)C3CCNCC3)s2)o1', 'CCOc1cc2ncc(C#N)c(Nc3ccc(OCc4ccccn4)c(Cl)c3)c2cc1NC(=O)C=CCN(C)C', 'Cc1[nH]c(C=C2C(=O)Nc3ccc(F)cc32)c(C)c1C(=O)NCC(O)CN1CCOCC1', 'Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4ccccc4)c3)cc12', 'CSc1cccc(Nc2ncc3cc(-c4c(Cl)cccc4Cl)c(=O)n(C)c3n2)c1', 'CCN1CCN(Cc2ccc(NC(=O)Nc3ccc(Oc4cc(NC)ncn4)cc3)cc2C(F)(F)F)CC1', 'CC1CCN(C(=O)CC#N)CC1N(C)c1ncnc2[nH]ccc12', 'Nc1nc(N)c2nc(-c3cccc(O)c3)c(-c3cccc(O)c3)nc2n1', 'N#CCC(C1CCCC1)n1cc(-c2ncnc3[nH]ccc23)cn1.O=P(O)(O)O']
['MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAEPKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITSRTQFNSLQQLVAYYSKHADGLCHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKG

In [6]:
from models.models import BiomedMultiViewMoleculeEncoder, T5ProstTargetEncoder

biomed = BiomedMultiViewMoleculeEncoder()
t5 = T5ProstTargetEncoder()

2024-11-17 15:07:57,472 - root - INFO - pop-os:133516169840192:0:0 - BioMed Multi-view embedding model
2024-11-17 15:07:57,473 - root - INFO - pop-os:133516169840192:0:0 - dim_list [512, 512, 768] of Graph2dModel, ImageModel, TextModel
2024-11-17 15:07:57,484 - root - INFO - pop-os:133516169840192:0:0 - Loading checkpoint from provided path ./models/bmfm_model_dir/biomed-smmv-base.pth
2024-11-17 15:07:57,622 - root - INFO - pop-os:133516169840192:0:0 - Loading pretrain checkpoint for SmallMoleculeMultiView Model - _IncompatibleKeys(missing_keys=['aggregator.projections.0.weight', 'aggregator.projections.0.bias', 'aggregator.projections.1.weight', 'aggregator.projections.1.bias', 'aggregator.projections.2.weight', 'aggregator.projections.2.bias', 'aggregator.w_before_mean.0.weight', 'aggregator.w_before_mean.0.bias', 'aggregator.w_before_mean.2.weight', 'aggregator.down_project.weight', 'aggregator.down_project.bias', 'aggregator.shared_task_head.0.weight', 'aggregator.shared_task_head.

In [7]:
print("Biomed multiview molecule encoder model")
result_biomed = biomed(batch["0/Drug_SMILES"])
for item in result_biomed:
    print(item.shape)

print("T5 protein target encoder model")
result_t5 = t5(batch["1/Target_seq"])
print(result_t5.shape)

Biomed multiview molecule encoder model
torch.Size([12, 512])
torch.Size([12, 512])
torch.Size([12, 768])
T5 protein target encoder model
torch.Size([12, 1024])


---
---

In [26]:
np.array(fp).shape

(2048,)

In [ ]:
from subword_nmt.apply_bpe import BPE
import codecs
import pandas as pd
import numpy as np

# See: https://github.com/kexinhuang12345/ESPF

espf_folder = "./data/ESPF"

vocab_path = espf_folder + '/codes_protein.txt'
bpe_codes_protein = codecs.open(vocab_path)
pbpe = BPE(bpe_codes_protein, merges=-1, separator='')
sub_csv = pd.read_csv(espf_folder + '/subword_units_map_protein.csv')

idx2word_p = sub_csv['index'].values
words2idx_p = dict(zip(idx2word_p, range(0, len(idx2word_p))))

s = "MGSNKSKPKDASQRRRSLEPAEQAFLEDYFTSTEPQYQPGENL"
t = pbpe.process_line(s).split()
i = [words2idx_p[i] for i in t]
v = np.zeros(len(idx2word_p), )
v[i] = 1